## This example demonstrates sklearn functions with XCSF

In [1]:
import json

from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from optuna.integration import OptunaSearchCV
import optuna
from optuna.distributions import FloatDistribution

import xcsf

RANDOM_STATE: int = 1

### Load some test data

In [2]:
iris = load_iris()

X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

### Initialise XCSF

In [3]:
def get_model():
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=4,
        y_dim=1,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=20000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.005,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction={
            "type": "neural",
            "args": {
                "layer_0": {
                    "type": "connected",
                    "activation": "relu",
                    "n_init": 10,
                    "evolve_weights": True,
                    "evolve_functions": False,
                    "evolve_connect": True,
                    "evolve_neurons": False,
                    "sgd_weights": True,
                    "eta": 0.1,
                    "evolve_eta": True,
                    "eta_min": 1e-06,
                    "momentum": 0.9,
                    "decay": 0,
                },
                "layer_1": {
                    "type": "connected",
                    "activation": "softplus",
                    "n_init": 1,
                    "evolve_weights": True,
                    "evolve_functions": False,
                    "evolve_connect": True,
                    "evolve_neurons": False,
                    "sgd_weights": True,
                    "eta": 0.1,
                    "evolve_eta": True,
                    "eta_min": 1e-06,
                    "momentum": 0.9,
                    "decay": 0,
                },
            },
        },
    )

In [4]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

In [5]:
parameters = {"beta": [0.1, 0.5]}


def objective(trial):
    """Measure a new hyperparameter combination."""

    beta = trial.suggest_float("beta", 0.1, 0.5)

    model = get_model()
    model.set_params(beta=beta)

    params = model.internal_params()
    print(f"beta={params['beta']}")

    callback = get_callback()

    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )

    return model.score(X_val, y_val)


study = optuna.create_study()
study.optimize(objective, n_trials=10)

[I 2023-07-17 15:49:53,095] A new study created in memory with name: no-name-79014d49-bf08-4d1c-970a-b07e57f1ca8b


beta=0.14395415563340805
time=00:00:00.344 trials=0 train=0.22003 val=0.30028 pset=500.00000 mset=293.8 mfrac=1.00
checkpoint: 0.30028 error at 0 trials
time=00:00:00.894 trials=5000 train=0.07788 val=0.03227 pset=500.00000 mset=326.7 mfrac=1.00
checkpoint: 0.03227 error at 5000 trials
time=00:00:00.723 trials=10000 train=0.05583 val=0.04993 pset=500.00000 mset=449.0 mfrac=1.00


[I 2023-07-17 15:49:55,871] Trial 0 finished with value: 0.03227214526766292 and parameters: {'beta': 0.14395415563340805}. Best is trial 0 with value: 0.03227214526766292.


time=00:00:00.791 trials=15000 train=0.04837 val=0.03671 pset=500.00000 mset=456.0 mfrac=1.00
restoring system from trial 5000 with error=0.03227
time=00:00:02.752
beta=0.1890095512038968
time=00:00:00.680 trials=0 train=0.19567 val=0.32069 pset=500.00000 mset=296.1 mfrac=1.00
checkpoint: 0.32069 error at 0 trials
time=00:00:01.272 trials=5000 train=0.08042 val=0.03013 pset=500.00000 mset=304.8 mfrac=1.00
checkpoint: 0.03013 error at 5000 trials
time=00:00:01.748 trials=10000 train=0.06615 val=0.03168 pset=500.00000 mset=413.8 mfrac=1.00


[I 2023-07-17 15:50:01,639] Trial 1 finished with value: 0.03013401109965067 and parameters: {'beta': 0.1890095512038968}. Best is trial 1 with value: 0.03013401109965067.


...

time=00:00:00.889 trials=15000 train=0.03585 val=0.06434 pset=500.00000 mset=353.8 mfrac=0.40
restoring system from trial 5000 with error=0.03608
time=00:00:04.078
beta=0.4254319658900404
time=00:00:01.159 trials=0 train=0.12779 val=0.06376 pset=500.00000 mset=238.6 mfrac=0.34
checkpoint: 0.06376 error at 0 trials
time=00:00:00.841 trials=5000 train=0.05055 val=0.03591 pset=500.00000 mset=214.3 mfrac=1.00
checkpoint: 0.03591 error at 5000 trials
time=00:00:01.446 trials=10000 train=0.03874 val=0.05622 pset=500.00000 mset=376.1 mfrac=0.45


[I 2023-07-17 15:50:35,919] Trial 8 finished with value: 0.034470714074061166 and parameters: {'beta': 0.4254319658900404}. Best is trial 1 with value: 0.03013401109965067.


time=00:00:01.610 trials=15000 train=0.04114 val=0.03447 pset=500.00000 mset=400.9 mfrac=0.46
checkpoint: 0.03447 error at 15000 trials
restoring system from trial 15000 with error=0.03447
time=00:00:05.056
beta=0.499600412705061
time=00:00:00.688 trials=0 train=0.10966 val=0.03047 pset=500.00000 mset=237.0 mfrac=1.00
checkpoint: 0.03047 error at 0 trials
time=00:00:00.871 trials=5000 train=0.04880 val=0.03231 pset=500.00000 mset=213.4 mfrac=1.00
time=00:00:01.506 trials=10000 train=0.04552 val=0.09821 pset=500.00000 mset=374.2 mfrac=0.30


[I 2023-07-17 15:50:40,363] Trial 9 finished with value: 0.03046689607147471 and parameters: {'beta': 0.49960041270506095}. Best is trial 1 with value: 0.03013401109965067.


time=00:00:01.355 trials=15000 train=0.04546 val=0.04840 pset=500.00000 mset=406.8 mfrac=0.28
restoring system from trial 0 with error=0.03047
time=00:00:04.420


### Pipeline

In [6]:
model = get_model()

model = make_pipeline(
    MinMaxScaler(feature_range=(-1.0, 1.0)),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

model.fit(X_train, y_train)

time=00:00:00.513 trials=0 train=0.55394 pset=500.00000 mset=276.8 mfrac=0.45
time=00:00:00.780 trials=5000 train=0.55222 pset=500.00000 mset=292.6 mfrac=0.44
time=00:00:00.838 trials=10000 train=0.49859 pset=500.00000 mset=351.7 mfrac=0.49
time=00:00:03.370 trials=15000 train=0.52357 pset=500.00000 mset=348.0 mfrac=0.45
time=00:00:05.501


Pipeline(steps=[('minmaxscaler', MinMaxScaler(feature_range=(-1.0, 1.0))),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7fdd6a0f1970>,
                                            transformer=StandardScaler()))])

In [7]:
y_pred = model.predict(X_test)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"train={train_score}")
print(f"test={test_score}")

train=0.47031717384027283
test=0.20063879503722692


### Grid search

General parameters can be searched in the usual way:

In [8]:
parameters = {"beta": [0.1, 0.5]}

model = get_model()

grid_search = GridSearchCV(model, parameters, scoring="neg_mean_squared_error")

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

time=00:00:00.467 trials=0 train=0.34194 pset=500.00000 mset=297.1 mfrac=1.00
time=00:00:01.411 trials=5000 train=0.09027 pset=500.00000 mset=332.7 mfrac=1.00
time=00:00:02.185 trials=10000 train=0.05746 pset=500.00000 mset=459.8 mfrac=1.00
time=00:00:01.836 trials=15000 train=0.05371 pset=500.00000 mset=444.0 mfrac=1.00
time=00:00:05.899
time=00:00:00.486 trials=0 train=0.21143 pset=500.00000 mset=291.1 mfrac=1.00
time=00:00:01.211 trials=5000 train=0.04051 pset=500.00000 mset=305.4 mfrac=1.00
time=00:00:01.686 trials=10000 train=0.03011 pset=500.00000 mset=447.4 mfrac=0.78
time=00:00:01.182 trials=15000 train=0.02480 pset=500.00000 mset=416.9 mfrac=0.40
time=00:00:04.565
time=00:00:00.429 trials=0 train=0.36399 pset=500.00000 mset=302.7 mfrac=0.62
time=00:00:01.075 trials=5000 train=0.13655 pset=500.00000 mset=302.4 mfrac=0.07
time=00:00:01.547 trials=10000 train=0.08819 pset=500.00000 mset=373.4 mfrac=0.11
time=00:00:01.506 trials=15000 train=0.05964 pset=500.00000 mset=378.8 mfrac=

EA parameters require specifying a dict, but individual values can still be set 
because the other values are still initialised to their default values.

In [9]:
parameters = {"ea": [{"lambda": 2}, {"lambda": 10}, {"lambda": 50}]}

However, for actions, conditions, and predictions, the WHOLE dict must be specified
for each value to try in the search. This is because of the way XCSF uses kwargs to
initialise values and they are reset each time. XCSF has so many different parameters
that it is unfortunately necessary to do it this way.

In [10]:
parameters = {}